#1. Projeto 2, Classificação multindex com pytorch


##2. Importando as bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import torch 
import torch.nn as nn

##3. Importando os dados

In [2]:
df = pd.read_csv('/content/iris.csv')
class_predict = df.pop('class')

##4. Transformando os dados em np array

In [3]:
entradas = np.array(df,dtype='float')
saidas_desejadas = class_predict.values

##5. Criando saida para multiclasses com probabilidade
1. Aqui vou fazer algo diferente, em vez de criar um modelo para prever uma classe, vou criar um pra prever a probabilidade entre as 3 classes, fazendo uma saidade de 3 classes, em vez de apenas 1

###5.1 Criando um dicionario para cada saida

In [4]:
saidas_unicas = np.unique(saidas_desejadas)
saidas = dict()
count = 1
for sd in saidas_unicas:
  saidas[sd] = count
  count = count+1

###5.2 Criando a saida 

In [5]:
saida_df = list()
for saida in saidas_desejadas:
  lis = list()
  for instancia in saidas:
    if saida == instancia:
      lis.append(1)
    else:
      lis.append(-1)
  saida_df.append(lis)
saida_df = np.array(saida_df)

##6. Normalizando as entradas

In [6]:
entradas_norm = MaxAbsScaler().fit(entradas)
entradas_norm = entradas_norm.transform(entradas)

##7. Definindo o fator randomico de reprodutibilidade para 1

In [7]:
np.random.seed(1)
torch.manual_seed(1)

##8. Criando a rede neural


In [8]:
class classificador(nn.Module):
  def __init__(self):#inicia o construtor
    super().__init__()
    self.camada0 = nn.Linear(in_features = 4, out_features = 5)
    torch.nn.init.uniform_(self.camada0.weight)
    self.ativacao0 = nn.ReLU()
    self.dp0 = nn.Dropout(0.2)
    self.camada1 = nn.Linear(in_features = 5, out_features = 5)
    torch.nn.init.uniform_(self.camada1.weight)
    self.ativacao1 = nn.ReLU()
    self.dp1 = nn.Dropout(0.2)
    self.camada2 = nn.Linear(in_features=5, out_features=3)
    torch.nn.init.uniform_(self.camada2.weight)
    self.output = nn.Softmax(dim=-1)
  
  def forward(self, x):
    x = self.camada0(x)
    x = self.ativacao0(x)
    x = self.dp0(x)
    x = self.camada1(x)
    x = self.ativacao1(x)
    x = self.dp1(x)
    x = self.camada2(x)
    x = self.output(x)
    return x

##9. Criando uma função de validação cruzada própria

In [110]:
class cross_Validation():
  def __init__(self,classificador,entrada,saida,cv):#inicia o construtor
    self.model = classificador
    self.x = entrada
    self.y = saida
    self.cv = cv
  
  def rede(self,classificador):
    return classificador()

  def split_data(self):
    self.x = np.array_split(self.x,self.cv)
    self.y = np.array_split(self.y,self.cv)
  
  def transform(self):
    self.split_data()
    self.x = torch.tensor(self.x,dtype=torch.float)
    self.y = torch.tensor(self.y,dtype=torch.long)
  
  def treinar(self,x,y):
    model = self.rede(self.model)
    er = nn.MultiLabelMarginLoss()
    otm = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay=0.0001)
    error = 0.0
    for epoch in range(100):
      ls = 0.0
      for npt,otp in zip(x,y):
        otm.zero_grad() 
        outputs = model.forward(npt) 
        loss = er(outputs,otp) #Calcula o erro
        loss.backward() #reajusta os pesos
        otm.step() #calcula o gradiente
        ls = ls+loss.item()# Calcula o erro global
      error = error + ls
    #error = error/len(x)
    return model

  def avaliar(self,x_treino,y_treino,x_teste,y_teste):
    lista = list()
    model = self.treinar(x_treino,y_treino)
    model.eval()
    prev = model.forward(x_teste)
    prev = prev.detach().numpy()
    for x in prev:
      for z in x:
        maior = np.argmax(z)
        for y in range(len(z)):
          if z[y] == z[maior]:
            z[y] = 1
          else:
            z[y] = -1
        lista.append(z)    
    '''
    for x in prev:
      req_index=np.argmax(x)
      for z in range(len(x)):
        if x[z] == x[req_index]:
          x[z] = 1
        else:
          x[z] = -1
      lista.append(x)
    '''
    lista = np.array(lista)
    return lista

  def conjunto(self,x,y,k):
    saida_treino = list()
    entrada_treino = list()
    saida_test = list()
    entrada_test = list()
    for indice in range(len(y)):
      if indice == k:
        saida_test.append(y[indice])
      else:
        saida_treino.append(y[indice]) 
    for indice in range(len(x)):
      if indice == k:
        entrada_test.append(x[indice])
      else:
        entrada_treino.append(x[indice])
    
    entrada_treino = torch.tensor(entrada_treino,dtype=torch.float)
    entrada_test = torch.tensor(entrada_test,dtype=torch.float)
    saida_treino = torch.tensor(saida_treino,dtype=torch.long)
    saida_test = torch.tensor(saida_test,dtype=torch.long)
    return entrada_treino,entrada_test,saida_test,saida_treino 

  def fit(self):
    self.transform()
    x_data = self.x.detach().numpy()
    y_data = self.y.detach().numpy()
    for k in range(self.cv):
      x_treino,x_teste,y_teste,y_treino = self.conjunto(x_data,y_data,k) 
      resposta = self.avaliar(x_treino,y_treino,x_teste,y_teste)
      for x,y in zip(resposta,y_teste):
        print('{} : {}'.format(x,y))

In [111]:
cl = cross_Validation(classificador=classificador,entrada=entradas_norm,saida=saida_df,cv=5)
#entradas_norm = torch.tensor(entradas_norm,dtype=torch.float)
#saida_df = torch.tensor(saida_df,dtype=torch.long)

In [112]:
cl.fit()

[-1.  1. -1.] : tensor([[ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1]])
[-1.  1. -1.] : tensor([[ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],
        [ 1, -1, -1],